In [1]:
import tkinter as tk
import re
import time

import ipynb_importer
from search_engine import Searcher

importing Jupyter notebook from search_engine.ipynb


In [2]:
class MainPanel:
    def __init__(self, searcher):
        self.searcher = searcher
        self.tot_subscribe = 0
        self.button2id = {}
    
    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.title("古诗订阅")
        tk.Label(self.root, text="我的订阅", font=(None,12)).grid(column = 1, row = 1)
        self.subscription_frame = tk.Frame(
            self.root, width = 400, height = 400, padx = 5, pady = 5)
        self.subscription_frame.pack_propagate(0)
        self.subscription_frame.grid(column = 1, row = 2)
        tk.Button(
            self.root, text="新增订阅", font=(None,12), command=self.add_subscription).grid(column = 1, row = 3, pady = 10)
        self.root.mainloop()
        
    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        tk.Label(self.add_panel, text="新增订阅:", font=(None, 12)).grid(column = 2, row = 1)
        tk.Label(self.add_panel, text = '作者', font = (None, 12)).grid(column = 1, row = 2)
        self.new_author_entry = tk.Entry(self.add_panel, font = (None, 12))
        self.new_author_entry.grid(column = 2, row = 2)
        
        tk.Label(self.add_panel, text = '关键词', font = (None, 12)).grid(column = 1, row = 3)
        self.new_keyword_entry = tk.Entry(self.add_panel, font = (None, 12))
        self.new_keyword_entry.grid(column = 2, row = 3)
        
        tk.Button(
            self.add_panel, text="确认", font=(None, 12), command=self.check_new_author).grid(column = 2, row = 4)
        
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.grid(column = 2, row = 5)
    
    def check_new_author(self):
        """
        用户点击"确认"按钮后，检查输入是否合法
        
        """
        new_author = self.new_author_entry.get()
        new_keyword = self.new_keyword_entry.get()
        
        if new_author ==  '' and new_keyword == '':
            self.hint_label.config(text = '订阅不能为空')
        else:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("200x150")
            tk.Label(tmp_tk, text="订阅成功", font=(None, 16)).pack(pady=10)
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=10)
            self.searcher.new_search(new_author, new_keyword)
            self.add_panel.destroy()
            self.update_main_panel(new_author, new_keyword)
            
    def update_main_panel(self, new_author, new_keyword):
        """
        用户新增订阅后，更新主界面
        """
        button = tk.Button(self.subscription_frame, text = new_author + ' ' + new_keyword, font=(None,12),
                     width = 40)
        button.bind("<Button-1>",self.show_poems)
        button.pack(pady = 10)
        self.button2id[button['text']] = self.tot_subscribe
        self.tot_subscribe += 1
        
    def show_poems(self, event):
        """
        用户点击主界面的“作者名-关键词”按钮，
        显示搜索结果
        
        """
        button_text = event.widget['text']
        self.subscribe_id = self.button2id[button_text]
        
        self.cur_page = 1
        self.poems = self.searcher.results[self.subscribe_id]
        self.tot_poems = len(self.poems)
        self.tot_pages = (self.tot_poems + 1) // 2
        
        self.poem_panel = tk.Tk()
        self.poem_panel.title('订阅内容')
        
        if self.tot_poems <= 0:
            tk.Label(self.poem_panel, text = '搜索结果为空', font = (None, 16)).pack()
            return
        
        self.txt = tk.Text(self.poem_panel, width = 30, height = 20, font = (None,14))
        self.txt.grid(column = 1, row = 1)
        self.txt.tag_configure("center", justify='center')
        
        self.page_frame = tk.Frame(self.poem_panel)
        self.page_frame.grid(column = 1, row = 2)
        
        self.update_page_labels()
        self.update_text()
        
    def update_text(self):
        """
        更新文本框
        """
        self.txt.delete(1.0, 'end')
        cur_poem = (self.cur_page - 1) * 2
        self.txt.insert('end', self.poems[cur_poem])
        self.txt.insert('end', '\n\n')
        if cur_poem + 1< self.tot_poems:
            self.txt.insert('end', self.poems[cur_poem + 1])
        for kw in self.searcher.keywords[self.subscribe_id]:
            self.highlight(self.txt, kw)
        self.txt.tag_add('center', 1.0, 'end')
            
    def highlight(self, Text, seq):
        #remove highlights
        #if "highlight" in Text.tag_names():
        #    Text.tag_delete("highlight")

        i = len(seq)
        index = "1.0"
        while True:
            index = Text.search(seq, index, stopindex='end')
            if index:
                index2 = Text.index("%s + %d chars" % (index, i))
                Text.tag_add("highlight", index, index2)
                Text.tag_config("highlight", foreground="blue")
                index = index2
            else: return
                    
    def update_page_labels(self):
        '''
        更新页码显示
        '''
        for widget in self.page_frame.winfo_children():
            widget.destroy()

        self.page_up_button = tk.Button(
            self.page_frame, text = '上一页', command = self.page_up)
        self.page_up_button.pack(side = tk.LEFT, anchor = tk.W)

        
        if self.cur_page != 1:
            self.add_page_label(1)
        if self.cur_page > 5:
            tk.Label(self.page_frame, text = '...').pack(side = tk.LEFT)
        for i in range(2, self.cur_page):
            if i >= self.cur_page - 3:
                self.add_page_label(i)

        tk.Label(self.page_frame, text = str(self.cur_page), fg = 'blue').pack(side = tk.LEFT)
        
        for i in range(self.cur_page + 1, self.tot_pages):
            if i <= self.cur_page + 3:
                self.add_page_label(i)
        if self.cur_page < self.tot_pages - 4:
            tk.Label(self.page_frame, text = '...').pack(side = tk.LEFT)
        if self.cur_page != self.tot_pages:
            self.add_page_label(self.tot_pages)
                
        self.page_down_button = tk.Button(
            self.page_frame, text = '下一页', command = self.page_down)
        self.page_down_button.pack(side = tk.LEFT, anchor = tk.E)
                
                    
    def add_page_label(self, page):
        '''
        添加页码标签
        '''
        lb = tk.Label(self.page_frame, text = str(page))
        lb.pack(side = tk.LEFT)
        lb.bind("<Button-1>",self.page_jump)
        lb.bind("<Enter>", lambda e: lb.configure(fg = 'green'))
        lb.bind("<Leave>", lambda e: lb.configure(fg = 'black'))
        
    def page_up(self):
        if self.cur_page > 1:
            self.cur_page -= 1
            self.update_page_labels()
            self.update_text()
    
    def page_down(self):
        if self.cur_page < self.tot_pages:
            self.cur_page += 1
            self.update_page_labels()
            self.update_text()
    
    def page_jump(self, event):
        button_text = event.widget['text']
        self.cur_page = int(button_text)
        self.update_page_labels()
        self.update_text()
        